# Data processing and visualizations

In [ ]:
import os
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir(os.pardir)

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
plt.rcParams.update({'text.usetex': True,
                     'text.latex.preamble': r'\usepackage{amsmath}',
                     'font.family': 'serif'})
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import seaborn as sns
sns.set(style='ticks', font='serif')

In [ ]:
PATH_RAW_DATA = os.path.join('data', 'raw')
PATH_PROC_DATA = os.path.join('data', 'processed')

try:
    clean_data = pd.read_csv(os.path.join(PATH_PROC_DATA, 'pDeltaT_clean.csv'))
except Exception as e:
    data = pd.read_excel(os.path.join(PATH_RAW_DATA, 'pDeltaT.xlsx'))
    clean_data = data.drop(columns='organization')
    clean_data.loc[:, 'pDeltaT * 100 [°C]'] = clean_data['pDeltaT [°C]'].values * 100
    clean_data.to_csv(os.path.join(PATH_PROC_DATA, 'pDeltaT_clean.csv'),
                      index=False)

In [ ]:
clean_data.describe()

In [ ]:
cs = sns.color_palette('rocket', 4)
ms = ['o', 'd', 's', '^']
fs = clean_data['f [GHz]'].unique()
ps = ['00', '01', '10', '11']


def exp_decay_func(x, a, b, c):
    return a * np.exp(-b * x) + c


def exp_fit(x, y, func=exp_decay_func):
    yavg = []
    yerr = []
    for _x in np.unique(x):
        _y = y[np.where(x==_x)]
        yavg.append(np.mean(_y))
        yerr.append(np.std(_y)/np.sqrt(_y.size))
    x = x[:len(yavg)]
    yavg = np.asarray(yavg)
    yerr = np.asarray(yerr)
    popt, pcov = curve_fit(func, x, yavg,
                           sigma=10/yerr, absolute_sigma=False,
                           bounds=([0, 0, -10], [1000, 1, 10]))
    perr = np.diag(pcov)
    return popt, perr


fig, axs = plt.subplots(2, 2, sharex=False, sharey=True, figsize=(4.5, 4))
for i, f in enumerate(fs):
    irow, icol = ps[i]
    irow, icol = int(irow), int(icol)
    
    # spatial domain
    d = clean_data[clean_data['f [GHz]'] == f]['d [mm]'].values
    _d = np.linspace(d.min(), d.max(), 101)
    
    # psPDn_4
    psPDn_4 = clean_data[clean_data['f [GHz]'] == f]['psPDn_4 [W/m2]'].values
    popt, perr = exp_fit(d, psPDn_4)
    axs[irow, icol].plot(d, psPDn_4, ms[0], c=cs[0])
    axs[irow, icol].plot(_d, exp_decay_func(_d, *popt), c=cs[0], lw=2, dashes=(1, 0))
    axs[irow, icol].fill_between(_d, color='b', alpha=0.2, 
                                 y1=exp_decay_func(_d, *(popt+3*perr)),
                                 y2=exp_decay_func(_d, *(popt-3*perr)))
    axs[irow, icol].plot([], [], ms[0], c=cs[0], dashes=(1, 0),
                         label=r'$psPD_{\text{n}, 4}$')
    
    # psPDtot_4
    psPDtot_4 = clean_data[clean_data['f [GHz]'] == f]['psPDtot_4 [W/m2]'].values
    popt, perr = exp_fit(d, psPDtot_4)
    axs[irow, icol].plot(d, psPDtot_4, ms[1], c=cs[1])
    axs[irow, icol].plot(_d, exp_decay_func(_d, *popt), c=cs[1], lw=2, dashes=(3, 1))
    axs[irow, icol].fill_between(_d, color=cs[1], alpha=0.2, 
                                 y1=exp_decay_func(_d, *(popt+3*perr)),
                                 y2=exp_decay_func(_d, *(popt-3*perr)))
    axs[irow, icol].plot([], [], ms[1], c=cs[1], dashes=(3, 1),
                         label=r'$psPD_{\text{tot}, 4}$')
    
    # psPDn_1
    psPDn_1 = clean_data[clean_data['f [GHz]'] == f]['psPDn_1 [W/m2]'].values
    popt, perr = exp_fit(d, psPDn_1)
    axs[irow, icol].plot(d, psPDn_1, ms[2], c=cs[2])
    axs[irow, icol].plot(_d, exp_decay_func(_d, *popt), c=cs[2], lw=2, dashes=(1, 1))
    axs[irow, icol].fill_between(_d, color=cs[2], alpha=0.2, 
                                 y1=exp_decay_func(_d, *(popt+3*perr)),
                                 y2=exp_decay_func(_d, *(popt-3*perr)))
    axs[irow, icol].plot([], [], ms[2], c=cs[2], dashes=(1, 1),
                         label=r'$psPD_{\text{n}, 1}$')
    
    # psPDtot_1
    psPDtot_1 = clean_data[clean_data['f [GHz]'] == f]['psPDtot_1 [W/m2]'].values
    popt, perr = exp_fit(d, psPDtot_1)
    axs[irow, icol].plot(d, psPDtot_1, ms[3], c=cs[3])
    axs[irow, icol].plot(_d, exp_decay_func(_d, *popt), c=cs[3], lw=2, dashes=(1, 3))
    axs[irow, icol].fill_between(_d, color=cs[3], alpha=0.2, 
                                 y1=exp_decay_func(_d, *(popt+3*perr)),
                                 y2=exp_decay_func(_d, *(popt-3*perr)))
    axs[irow, icol].plot([], [], ms[3], c=cs[3], dashes=(1, 3),
                         label=r'$psPD_{\text{tot}, 1}$')
    
    # settings
    axs[irow, icol].set(xscale='log',
                        xticks=clean_data['d [mm]'].unique(),
                        xticklabels=clean_data['d [mm]'].unique(),
                        yticks=[0, 40, 80],
                        yticklabels=[0, 40, 80],
                        ylim=[-5, 85])
    axs[irow, icol].set_title(f'${f}$ GHz')

fig.supxlabel('$d$ [mm]')
fig.supylabel('incident power density [W/m$^2$]')
fig.suptitle(r'$\mathbf{(a)}$', x=0.08, y=0.92)
handles, labels = fig.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(),
           bbox_to_anchor=(1.275, 0.625))
fig.tight_layout()
sns.despine()

# fig_name = os.path.join('figures', 'ipd.png')
# fig.savefig(fig_name, dpi=200, bbox_inches='tight')

In [ ]:
cs = sns.color_palette('rocket', 1)
fs = clean_data['f [GHz]'].unique()
ps = ['00', '01', '10', '11']

fig, axs = plt.subplots(2, 2, sharex=False, sharey=True, figsize=(4.5, 4))
for i, f in enumerate(fs):
    irow, icol = ps[i]
    irow, icol = int(irow), int(icol)
    
    # spatial domain
    d = clean_data[clean_data['f [GHz]'] == f]['d [mm]'].values
    
    # temperature rise, avg, min & max
    pDeltaT = clean_data[clean_data['f [GHz]'] == f]['pDeltaT [°C]'].values
    pDeltaT_avg = []
    pDeltaT_min = []
    pDeltaT_max = []
    for _d in np.unique(d):
        pDeltaT_avg.append(np.mean(pDeltaT[np.where(d==_d)]))
        pDeltaT_min.append(np.min(pDeltaT[np.where(d==_d)]))
        pDeltaT_max.append(np.max(pDeltaT[np.where(d==_d)]))
    dstrip = d[:len(pDeltaT_avg)]
    axs[irow, icol].errorbar(dstrip, pDeltaT_avg,
                             yerr=np.c_[pDeltaT_min, pDeltaT_max].T,
                             marker='o', color=cs[0], lw=2, capsize=3)
    
    # settings
    axs[irow, icol].set(xscale='log',
                        xticks=clean_data['d [mm]'].unique(),
                        xticklabels=clean_data['d [mm]'].unique(),
                        yticks=[0, 1, 2],
                        yticklabels=[0, 1, 2],
                        ylim=[-0.1, 2.1])
    axs[irow, icol].set_title(f'${f}$ GHz')

fig.supxlabel('$d$ [mm]')
fig.supylabel(r'$\Delta T_\text{max}$ [°C]')
fig.suptitle('$\\mathbf{(b)}$', x=0.08, y=0.92)
fig.tight_layout()
sns.despine()

# fig_name = os.path.join('figures', 'temp.png')
# fig.savefig(fig_name, dpi=200, bbox_inches='tight')

In [ ]:
cs = sns.color_palette('rocket', 4)
ms = ['o', 'd', 's', '^']
As = [4, 4, 1, 1]
fs = clean_data['f [GHz]'].unique()
ds = [(1, 0), (3, 1), (1, 1), (1, 3)]

fig, axs = plt.subplots(2, 2, sharex=False, sharey=True, figsize=(4.5, 4))
for i, f in enumerate(fs):
    # psPDn_4
    axs[0, 0] = sns.regplot(clean_data[clean_data['f [GHz]'] == f],
                            x='psPDn_4 [W/m2]', y='pDeltaT [°C]',
                            marker=ms[i], color=cs[i], ax=axs[0, 0],
                            line_kws={'linewidth': 2, 'dashes': ds[i]})
    axs[0, 0].plot([], [], ms[i], c=cs[i], lw=2, dashes=ds[i], label=f'${f}$')
    axs[0, 0].set(title=r'$psPD_{\text{n}, 4}$ [W/m$^2$]',
                  xlabel='', ylabel='',
                  xticks=[0, 7, 14],
                  xticklabels=[0, 7, 14],
                  xlim=[-0.5, 14.5],
                  yticks=[0, 0.7, 1.4],
                  yticklabels=[0, 0.7, 1.4],
                  ylim=[-0.1, 1.5])
    
    # psPDtot_4
    axs[0, 1] = sns.regplot(clean_data[clean_data['f [GHz]'] == f],
                            x='psPDtot_4 [W/m2]', y='pDeltaT [°C]',
                            marker=ms[i], color=cs[i], ax=axs[0, 1],
                            line_kws={'linewidth': 2, 'dashes': ds[i]})
    axs[0, 1].plot([], [], ms[i], c=cs[i], lw=2, dashes=ds[i], label=f'${f}$')
    axs[0, 1].set(title=r'$psPD_{\text{tot}, 4}$ [W/m$^2$]',
                  xlabel='', ylabel='',
                  xticks=[0, 16, 32],
                  xticklabels=[0, 16, 32],
                  xlim=[-1, 33],
                  yticks=[0, 0.7, 1.4],
                  yticklabels=[0, 0.7, 1.4],
                  ylim=[-0.1, 1.5])
    
    # psPDn_1
    axs[1, 0] = sns.regplot(clean_data[clean_data['f [GHz]'] == f],
                            x='psPDn_1 [W/m2]', y='pDeltaT [°C]',
                            marker=ms[i], color=cs[i], ax=axs[1, 0],
                            line_kws={'linewidth': 2, 'dashes': ds[i]})
    axs[1, 0].plot([], [], ms[i], c=cs[i], lw=2, dashes=ds[i], label=f'${f}$')
    axs[1, 0].set(title=r'$psPD_{\text{n}, 1}$ [W/m$^2$]',
                  xlabel='', ylabel='',
                  xticks=[0, 20, 40],
                  xticklabels=[0, 20, 40],
                  xlim=[-2.5, 42.5],
                  yticks=[0, 0.7, 1.4],
                  yticklabels=[0, 0.7, 1.4],
                  ylim=[-0.1, 1.5])
    
    # psPDtot_1
    axs[1, 1] = sns.regplot(clean_data[clean_data['f [GHz]'] == f],
                            x='psPDtot_1 [W/m2]', y='pDeltaT [°C]',
                            marker=ms[i], color=cs[i], ax=axs[1, 1],
                            line_kws={'linewidth': 2, 'dashes': ds[i]})
    axs[1, 1].plot([], [], ms[i], c=cs[i], lw=2, dashes=ds[i], label=f'${f}$')
    axs[1, 1].set(title=r'$psPD_{\text{tot}, 1}$ [W/m$^2$]',
                  xlabel='', ylabel='',
                  xticks=[0, 35, 70],
                  xticklabels=[0, 35, 70],
                  xlim=[-3, 74],
                  yticks=[0, 0.7, 1.4],
                  yticklabels=[0, 0.7, 1.4],
                  ylim=[-0.1, 1.5])
    
fig.supxlabel('incident power density [W/m$^2$]')
fig.supylabel(r'$\Delta T_{max}$ [°C]')
fig.suptitle('$\\mathbf{(a)}$', x=0.08, y=0.92)
handles, labels = fig.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(),
           title='$f$ [GHz]',
           bbox_to_anchor=(1.175, 0.675))
fig.tight_layout()
sns.despine()

# fig_name = os.path.join('figures', 'corr.png')
# fig.savefig(fig_name, dpi=200, bbox_inches='tight')

In [ ]:
cs = sns.color_palette('rocket', 2)
ms = ['o', 'd', 's', '^']
As = [4, 4, 1, 1]
fs = clean_data['f [GHz]'].unique()
ps = ['00', '01', '10', '11']

fig, axs = plt.subplots(2, 2, sharex=False, sharey=True, figsize=(4.5, 4))
for i, f in enumerate(fs):
    irow, icol = ps[i]
    irow, icol = int(irow), int(icol)
    
    # spatial domain
    d = clean_data[clean_data['f [GHz]'] == f]['d [mm]'].values
    
    # incident EM power and temperature rise
    if f <= 30:  # A = 4 cm2
        psPDn = clean_data[clean_data['f [GHz]'] == f]['psPDn_4 [W/m2]'].values
        psPDtot = clean_data[clean_data['f [GHz]'] == f]['psPDtot_4 [W/m2]'].values
    else:  # A = 1 cm2
        psPDn = clean_data[clean_data['f [GHz]'] == f]['psPDn_1 [W/m2]'].values
        psPDtot = clean_data[clean_data['f [GHz]'] == f]['psPDtot_1 [W/m2]'].values
    pDeltaT = clean_data[clean_data['f [GHz]'] == f]['pDeltaT [°C]'].values
    
    # compute heating factors, avg, min & max
    HFn = pDeltaT / psPDn
    HFtot = pDeltaT / psPDtot
    HFn_avg, HFtot_avg = [], []
    HFn_min, HFtot_min= [], []
    HFn_max, HFtot_max = [], []
    for _d in np.unique(d):
        HFn_avg.append(np.mean(HFn[np.where(d==_d)]))
        HFn_min.append(np.min(HFn[np.where(d==_d)]))
        HFn_max.append(np.max(HFn[np.where(d==_d)]))
        HFtot_avg.append(np.mean(HFtot[np.where(d==_d)]))
        HFtot_min.append(np.min(HFtot[np.where(d==_d)]))
        HFtot_max.append(np.max(HFtot[np.where(d==_d)]))
    dstrip = d[:len(HFn_avg)]
    axs[irow, icol].errorbar(dstrip, HFn_avg,
                             yerr=np.c_[HFn_min, HFn_max].T,
                             marker=ms[0], c=cs[0], lw=2, capsize=3,
                             label=r'$HF_{\text{n}}$')
    axs[irow, icol].errorbar(dstrip, HFtot_avg,
                             yerr=np.c_[HFtot_min, HFtot_max].T,
                             marker=ms[1], c=cs[1], lw=2, dashes=(3, 1), capsize=3,
                             label=r'$HF_{\text{tot}}$')
    
    # settings
    axs[irow, icol].set(xscale='log',
                        xticks=clean_data['d [mm]'].unique(),
                        xticklabels=clean_data['d [mm]'].unique())
    if f <= 30:
        axs[irow, icol].set(yticks=[0, 0.08, 0.16],
                            yticklabels=[0, 0.08, 0.16],
                            ylim=[-0.01, 0.17])
    axs[irow, icol].set_title(f'${f}$ GHz \t $(A = {As[i]}$ cm$^2)$')
    
fig.supxlabel('$d$ [mm]')
fig.supylabel('heating factor [W/m$^2$]')
fig.suptitle(r'$\mathbf{(b)}$', x=0.08, y=0.92)
handles, labels = fig.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(),
           bbox_to_anchor=(1.225, 0.575))
fig.tight_layout()
sns.despine()

# fig_name = os.path.join('figures', 'hf.png')
# fig.savefig(fig_name, dpi=200, bbox_inches='tight')